In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time

In [2]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

lower = 0
upper = 1

In [3]:
def auto_canny(image, sigma=0.33):
    # compute the median of the single channel pixel intensities
    v = np.median(image)

    # apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)

    # return the edged image
    return edged

In [4]:
MIN_MATCH_COUNT=30
madfox = cv2.imread('madfox.png', cv2.IMREAD_GRAYSCALE)

detector = cv2.xfeatures2d.SIFT_create()
madfox = cv2.imread('madfox.png', cv2.IMREAD_GRAYSCALE)
kpmad, des1=detector.detectAndCompute(madfox,None) 

FLANN_INDEX_KDITREE=0
flannpar = dict(algorithm=FLANN_INDEX_KDITREE,tree=5)
flann = cv2.FlannBasedMatcher(flannpar, {})

In [5]:
while(True):
    # Capture frame-by-frame
    print("New frame")
    ret, frame = cap.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # A gaussian blur to get rid of the noise in the image
    blur = cv2.GaussianBlur(gray,(5,5),0)
    #blur = gray
    # Detect the edges present in the image
    #bordas = auto_canny(blur)
    bordas = blur
    kpgray, des2=detector.detectAndCompute(blur,None)
    matches=flann.knnMatch(des1,des2,k=2)
    Pcorrespondentes = []
    for m, n in matches:
        if m.distance < 0.7*n.distance:
            Pcorrespondentes.append(m)

    if len(Pcorrespondentes)>MIN_MATCH_COUNT:
        
            src_pts = np.float32([ kpmad[m.queryIdx].pt for m in Pcorrespondentes ]).reshape(-1,1,2)
            dst_pts = np.float32([ kpgray[m.trainIdx].pt for m in Pcorrespondentes ]).reshape(-1,1,2)
            M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
            matchesMask = mask.ravel().tolist()
            
            
            h,w=madfox.shape
            pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)

            dst = cv2.perspectiveTransform(pts,M)
            img2b = cv2.polylines(blur,[np.int32(dst)],True,255,3, cv2.LINE_AA)
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(img2b,'Mad Fox',(300,50), font, 1.5,(255,255,255),2,cv2.LINE_AA)

            
            
    else:
        print("Not enough matches are found - %d/%d" % (len(Pcorrespondentes),MIN_MATCH_COUNT))
        matchesMask = None
        


    draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)
#//////////////////////////////////////////////////////////////////////////////////////////////////////////////
    circles = []


    # Obtains a version of the edges image where we can draw in color
    bordas_color = cv2.cvtColor(bordas, cv2.COLOR_GRAY2BGR)

    # HoughCircles - detects circles using the Hough Method. For an explanation of
    # param1 and param2 please see an explanation here http://www.pyimagesearch.com/2014/07/21/detecting-circles-images-using-opencv-hough-circles/
    circles = None
    circles=cv2.HoughCircles(bordas,cv2.HOUGH_GRADIENT,2,40,param1=50,param2=100,minRadius=10,maxRadius=60)

    if circles is not None:
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(bordas_color,'Circulo',(100,50), font, 1.5,(255,255,255),2,cv2.LINE_AA)
        circles = np.uint16(np.around(circles))

        for i in circles[0,:]:
            print(i)
            # draw the outer circle
            # cv2.circle(img, center, radius, color[, thickness[, lineType[, shift]]])
            cv2.circle(bordas_color,(i[0],i[1]),i[2],(0,255,0),2)
            # draw the center of the circle
            cv2.circle(bordas_color,(i[0],i[1]),2,(0,0,255),3)

#//////////////////////////////////////////////////////////////////////////////////////////////////////////////

    #More drawing functions @ http://docs.opencv.org/2.4/modules/core/doc/drawing_functions.html

    # Display the resulting frame
    cv2.imshow('Detector de circulos',bordas_color)
    #cv2.imshow('sfaf' , img2b )
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

New frame
[253 169  23]
[359 195  42]
New frame
[237 239  24]
[353 251  38]
New frame
[235 251  27]
[347 257  32]
New frame
[237 253  24]
[359 269  41]
New frame
[237 253  26]
[355 267  38]
New frame
[245 249  24]
[357 251  32]
New frame
[247 243  24]
[359 247  34]
New frame
[245 241  27]
[357 243  32]
New frame
[241 237  27]
[365 253  48]
New frame
[243 231  26]
[349 229  30]
New frame
[247 221  23]
[357 221  32]
New frame
[245 199  23]
New frame
New frame
[255 147  26]
[371 157  34]
New frame
[255 133  29]
[375 147  36]
New frame
[259 133  25]
New frame
[257 133  27]
[377 153  44]
New frame
[247 143  24]
[363 153  36]
New frame
[241 145  25]
New frame
[239 147  23]
[397 149  53]
New frame
[227 145  26]
[341 159  34]
New frame
[231 135  23]
New frame
[251 145  25]
New frame
[259 193  23]
New frame
[257 217  27]
[369 229  32]
New frame
[253 219  23]
[375 245  48]
New frame
[249 221  27]
[371 247  52]
New frame
[251 221  26]
[363 235  34]
[327 415  44]
New frame
[253 219  23]
[323 411  

[233 171  36]
[357 175  34]
[317 167  46]
New frame
[235 175  33]
[303 177  59]
New frame
[239 185  32]
[311 175  52]
[359 175  32]
[303 217  36]
New frame
[243 195  33]
[307 173  56]
[313 231  39]
New frame
[241 201  34]
[315 179  50]
[309 219  39]
New frame
[249 209  32]
[307 209  36]
New frame
[249 211  32]
[303 219  47]
[313 171  50]
New frame
[235 207  34]
[303 215  36]
New frame
[217 203  30]
[351 177  40]
[293 213  42]
New frame
[199 201  31]
[301 163  46]
[291 229  48]
New frame
Not enough matches are found - 28/30
[185 201  31]
[289 239  43]
New frame
[177 203  31]
[287 147  52]
[283 211  36]
New frame
[173 203  31]
[281 251  51]
[327 161  40]
New frame
[191 203  33]
New frame
[223 201  30]
New frame
[233 199  32]
[323 165  40]
New frame
Not enough matches are found - 12/30
[231 203  35]
[315 171  33]
[317 419  51]
New frame
[229 203  35]
[309 167  40]
New frame
Not enough matches are found - 27/30
[229 201  32]
[309 175  50]
New frame
Not enough matches are found - 30/30
[227

error: C:\bld\opencv_1498157567167\work\opencv-3.2.0\modules\core\src\matmul.cpp:2271: error: (-215) scn + 1 == m.cols in function cv::perspectiveTransform


In [ ]:
cap.release()
cv2.destroyAllWindows()